Based on the github repository by Koutny et all at https://github.com/dkoutny/DeepEntanglement

In [ ]:
from numpy import *
from numpy import linalg as la
from scipy import linalg as sa

import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
import pandas as pd

In [ ]:
def mubpom():
    p1 = array([1,0])
    p2 = array([0,1])
    mub = zeros([6,1,2])+1j*zeros([6,1,2])
    mub[0] = p1
    mub[1] = p2
    mub[2] = 1/sqrt(2)*(p1+p2)
    mub[3] = 1/sqrt(2)*(p1-p2)
    mub[4] = 1/sqrt(2)*(p1+1j*p2)
    mub[5] = 1/sqrt(2)*(p1-1j*p2)
    mubp = [transpose(mub[i])@conjugate(mub[i]) for i in range(6)]
    return mubp

def pauliK(data1,exc):
    """function pauliK takes full probability vector data1 and number of excluded projections exc and returns"""
    """probability vector data and projections with zeros in the appropriate places"""
    data = data1/sum(data1)
    mub = mubpom()
    mubP = array([kron(mub[i],mub[j])/9 for i in range(6) for j in range(6) ])
    myList=list(range(36))
    random.shuffle(myList)
    for i in myList[0:exc]:
        mubP[i] = zeros([4,4],complex)
        data[i] = 0
    return data,mubP

def concM(rrho):
    """Return the concurrence of a density matrix."""
    sigma2y = array([[0, 0, 0, -1], [0, 0, 1, 0], [0, 1, 0, 0], [-1, 0, 0, 0]])
    rho_spinflipped = sigma2y @ rrho.conj() @ sigma2y
    R = sa.sqrtm(sa.sqrtm(rrho) @ rho_spinflipped @ sa.sqrtm(rrho))
    R_eigen = la.eig(R)[0]
    R_eigen = sort(R_eigen)[::-1]
    concurrence = max([0, R_eigen[0] - sum(R_eigen[1:])])
    # check_imag(concurrence)
    return abs(concurrence)

def MLalg(data, rhoinit,povm,epsilon,stop):
    """Regularized maximum likelihood iterative algorithm"""
    rho = rhoinit
    trdist = 1
    k = 0
    pomS = zeros([4,4], complex)
    for n in range(shape(povm)[0]): pomS = pomS + povm[n]
#     Gop = sa.fractional_matrix_power(pomS,-1/2)
    Gop = sa.pinv(sa.fractional_matrix_power(pomS,1/2))
    while trdist > epsilon and k<stop:
        R = zeros([4,4], complex)
        prob = probdists(rho,povm)
        for i in range(shape(povm)[0]):
            if data[i]>10**(-10):
                R += data[i]/prob[i]*povm[i]
        rhonew = dot(dot(Gop,dot(R,rho)),dot(R,Gop))
        rhonew = rhonew/trace(rhonew)
        trdist = trace(dot((rho-rhonew),(rho-rhonew)))
        rho = rhonew
        k+=1
    return rho
def generate_werner_states(n):
    """
    Generates an array of Werner states with the parameter p varying from 0.0 to 1.0 in steps of 0.1.
    """
    # Define the Bell state |Psi_minus> in a 4x4 density matrix form
    psi_minus = array([[0, 0, 0, 0], [0, 1, -1, 0], [0, -1, 1, 0], [0, 0, 0, 0]]) / 2
    identity_matrix = identity(4)

    # Create Werner states
    p_values = linspace(0, 1, n)  # Linearly spaced values for p
    werner_states = [p * psi_minus + (1 - p)/4 * identity_matrix for p in p_values]

    return array(werner_states)

In [ ]:
mub = mubpom()
mub2 = array([kron(mub[i],mub[j])/9 for i in range(6) for j in range(6)])

**Datos Werner:**

In [ ]:
nR=0
dim=4
def nMubs(nProj):
    mubsN = linspace(0,35,nProj,dtype=int)
    mub = mubpom()
    mub2 = array([kron(mub[i],mub[j])/9 for i in range(6) for j in range(6)])
    mub3 = array([ mub2[n] for n in mubsN ])
    return mub3

def probdists(rho0, povm):
  """Get probabilities from quantum state rho0 and POVM."""
  from numpy import shape, array, real, trace, sum

  l = shape(povm)[0]
  probtrue = array([real(trace(rho0@povm[i])) for i in range(l)])
  probtrue = probtrue/sum(probtrue)
  return probtrue
def generate_werner_states(n):
    """
    Generates an array of Werner states with the parameter p varying from 0.0 to 1.0 in steps of 0.1.
    """
    # Define the Bell state |Psi_minus> in a 4x4 density matrix form
    psi_minus = array([[0, 0, 0, 0], [0, 1, -1, 0], [0, -1, 1, 0], [0, 0, 0, 0]]) / 2
    identity_matrix = identity(4)

    # Create Werner states
    p_values = linspace(0, 1, n)  # Linearly spaced values for p
    werner_states = [p * psi_minus + (1 - p)/4 * identity_matrix for p in p_values]

    return array(werner_states)
def stokes_from_rho(rho, A):
    from numpy import shape, array, real, trace

    l = shape(A)[0]
    return array([real(trace(rho @ A[n])) for n in range(l)])
def gellmann(Q, dim):
    from numpy import zeros, trace, sqrt

    q = zeros([dim**2, dim, dim]) + 1j * zeros([dim**2, dim, dim])
    for i in range(dim**2):
        v = Q[i]
        for j in range(0, i):
            v = v - trace(v @ q[j]) * q[j]
        q[i] = v / sqrt(trace(v @ v))
    return q
def herbasis(dim):
    from numpy import zeros, identity, dot, transpose, stack, concatenate

    pom1 = zeros([1, dim, dim]) + 1j * zeros([1, dim, dim])
    pom1[0] = identity(dim)
    arrays = [dot(transpose(pom1[0][[i]]), pom1[0][[i]]) for i in range(dim - 1)]
    pom = stack(arrays, axis=0)
    her = concatenate((pom1, pom), axis=0)
    arrays = [
        dot(transpose(her[0][[i]]), her[0][[j]])
        + dot(transpose(her[0][[j]]), her[0][[i]])
        for i in range(dim)
        for j in range(i + 1, dim)
    ]
    pom = stack(arrays, axis=0)
    her = concatenate((her, pom), axis=0)
    arrays = [
        -1j * dot(transpose(her[0][[i]]), her[0][[j]])
        + 1j * dot(transpose(her[0][[j]]), her[0][[i]])
        for i in range(dim)
        for j in range(i + 1, dim)
    ]
    pom = stack(arrays, axis=0)
    pom = concatenate((her, pom), axis=0)
    return pom

#--------------------------------------------------------------------------------------------------------------
#Measurement Specific

ks=[4,9,14,18]
tablaFinal=[]

quantity=11
dataWS = generate_werner_states(quantity)

for k in ks:
    nProj=2*k

    dictBestModel={}
    dictBestModel[str(k)]='/kaggle/input/specdnnsmodels/best_model_concurrence_R0_'+str(2*k)
    model = load_model(dictBestModel[str(k)]+'.keras',compile=False)


    file1 = load('/kaggle/input/specdnnsmodels/mub3_'+str(2*k)+".npz")
    mub3 = file1['mub3']
    srmPOMs = array([ mub3 for k in range(quantity) ])

    probListSRM = array([ probdists(dataWS[n],srmPOMs[n]) for n in range(quantity)])
    tabla=model.predict(probListSRM)
    tablaFinal.append(tabla)


tablaFinalMS=hstack(tablaFinal)
savetxt('concWsR0v7.txt', tablaFinalMS, fmt='%s')

#--------------------------------------------------------------------------------------------------------------
#Measurement Independent

tablaFinalMI=[]
modelMI = load_model('/kaggle/input/independant-model/bestModelConcHaar.tf',compile=False)

Q = herbasis(dim)
G_all = gellmann(Q, dim) * sqrt(dim)
G = G_all[1::]

stokesFromPOMs = array([array([stokes_from_rho(srmPOMs[n][m], G_all) for m in range(nProj)])for n in range(quantity)])

ksMI=[18,14,9,4]
for k in ksMI:
    nProjMI=2*k
    
    file2 = load('/kaggle/input/specdnnsmodels/mub3_'+str(2*k)+".npz")
    
    mub3MI = file2['mub3']
    srmPOMsMI = array([ mub3MI for k in range(quantity) ])
    
    probListSRM_MI = array([ probdists(dataWS[n],srmPOMsMI[n]) for n in range(quantity)])
    x = zeros([int(quantity), 17 * 36, 1])
    for n in range(int(quantity)):
        for i in range(nProjMI):
            x[n, i * 17 : i * 17 + 16, 0] = stokesFromPOMs[n, i]
            x[n, i * 17 + 16, 0] = probListSRM[n, i]
            
    tablaMI=modelMI.predict(x)
    tablaFinalMI.append(tablaMI)

tablaFinalMI=hstack(tablaFinalMI)
savetxt('netPredWsSv9.txt', tablaFinalMI, fmt='%s')

**Plots Werner:**

In [ ]:
"""concurrence of the Werner state"""

xx = linspace(1,11,11,dtype=int)
x = linspace(0,1,100)
# concTrue = array([max(0,(3*n-1)/2) for n in x])

# netSpecConc0 = loadtxt('/kaggle/input/data-for-figures/data_for_figures/WsState/concWsR0v7.txt')
# netSpecConc1 = loadtxt('/kaggle/input/data-for-figures/data_for_figures/WsState/concWsR1v7.txt')
# netSpecConc2 = loadtxt('/kaggle/input/data-for-figures/data_for_figures/WsState/concWsR2v7.txt')
# netSpecConc3 = loadtxt('/kaggle/input/data-for-figures/data_for_figures/WsState/concWsR3v7.txt')
# netSpecConc4 = loadtxt('/kaggle/input/data-for-figures/data_for_figures/WsState/concWsR4v7.txt')


# netSC = dstack((netSpecConc0,netSpecConc1,netSpecConc2,netSpecConc3,netSpecConc4))
# netSCMean = mean(netSpecConc0,axis=0)
# netSCStd = std(netSC,axis=2)

# netEvalConcM = loadtxt('/kaggle/input/data-for-figures/data_for_figures/WsState/netPredWsMv9.txt')
# netEvalConcS = loadtxt('/kaggle/input/data-for-figures/data_for_figures/WsState/netPredWsSv9.txt')
conMLwSM = loadtxt('/kaggle/input/data-for-figures/data_for_figures/WsState/mlPredWsMean.txt')
# conMLwSS = loadtxt('/kaggle/input/data-for-figures/data_for_figures/WsState/mlPredWsStd.txt')

#Our predictions:
netSCMean = loadtxt('/kaggle/working/concWsR0v7.txt')
netEvalConcM = loadtxt('/kaggle/working/netPredWsSv9.txt')



"""k = 0,1,2,3 corresponds with 36,28,18,8 measurement settings used to estimate concurrence of the Werner state"""

kk = 3

concTrue = array([max(0,(3*n-1)/2) for n in x])

plt.figure(figsize=(5.5, 4.5))

# plt.fill_between(xx,conMLwSM[:,kk]+conMLwSS[:,kk],conMLwSM[:,kk]-conMLwSS[:,kk],alpha=0.15,color ='red',linewidth=0.0)
# plt.fill_between(xx,netEvalConcM[:,kk]+netEvalConcS[:,kk],netEvalConcM[:,kk]-netEvalConcS[:,kk],alpha=0.15,color ='blue',linewidth=0.0)
# plt.fill_between(xx,netSCMean[:,3-kk]+netSCStd[:,3-kk],netSCMean[:,3-kk]-netSCStd[:,3-kk],alpha=0.15,color ='green',linewidth=0.0)

plt.plot(xx,conMLwSM[:,kk],'^-',color = 'red',label='RML')
plt.plot(xx,netEvalConcM[:,kk],'s-',color = 'blue',label='Device indep.')
plt.plot(xx,netSCMean[:,3-kk],'o-',color = 'green',label='Specific projs')
plt.plot(x*10+1,concTrue,'-',color = 'brown',linewidth="3.0",label='true val.')


# plt.xlabel(r'p',fontsize=12)
plt.xticks(array([ 1, 3, 5, 7, 9, 11]),array([0.0, 0.2, 0.4, 0.6, 0.8, 1. ]))


plt.tick_params(axis='both', which='major', labelsize=22)
plt.xlim([1,11])
plt.ylim([0,1])

# plt.savefig(('Werner36Proj.png'))